# HOMEWORK 3

In [1]:
!pip -q install controlSBML
import controlSBML as ctl
import control
from controlSBML.util import makeSimulationTimes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
import tellurium as te
import sympy
print("controlSBML version: " + ctl.__version__)

controlSBML version: 0.2.10


<img src="https://github.com/joseph-hellerstein/advanced-controls-lectures/blob/main/Lecture_13_14-Closed-Loop-Systems/Closed-Loop-System.png?raw=true"
     alt="Markdown Monster icon" width=600pt
     style="float: left; margin-right: 10px;" />

In this homework, you will design a closed loop system for controlling the mTOR system as modelled in BioModels 823.
1. (4 pt) Develop a "testbed factory" for evaluating a control designs.
    1. Develop a function ``makeTestbed`` that returns a ``control.Interconnect`` object for the testbed.
        * Inputs: kp, ki, kd, kf, noise_amp, noise_frq, disturbance_amp, disturbance_frq
        * Outputs: control.Interconnect
    1. Develop a function ``evaluateTestbed`` that plots the output of the closed loop system (y(t) + n(t))
1. (3 pt) Develop a function ``designClosedLoopSystem`` that searches for $k_P$, $k_I$, $k_D$, $k_F$ such that:
    1. All poles are less than -1;
    1. The DC gain of $H_{RY} \approx 1$;
    1. The DC gain for $H_NY$, $H_DY$ is less than 0.1.
1. (2 pt) Evaluate candidate design parameters from (2) on the testbed in (1) by showing plots of step response.
1. (1 pt) Discusses issues/challenges that you encountered in developing (2) and going from (2) to (3).

# Q1. ``evaluateTestbed``

# Q2. ``designClosedLoopSystem``

# Q3. Design Evaluation

# Q4. Discussion